## Column Consolidation

The aim of this notebook is to find groupings of column_names that refer to the same entity across different datasets. The approach we have taken is:
1. Extract a list of unique column names and their frequency of occurrence
2. Calculate a the similarity between every two column names, based on Levenshtein distance, stored in a matrix
3. Run k-means clustering on the column names, using the similarity of a column name to every other column names (each row in the similarity matrix) as features, and using the column name frequency as weights
4. Identify clusters with low average distance from the cluster centroids and inspect these clusters as groups of column names that refer to the same entity

In [5]:
## Install dependencies
!pip install pandas
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install sklearn

In [1]:
import os
import json
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from sklearn.cluster import KMeans

In [7]:
class FuzzMatrix:
    """
    A wrapper around a similarity matrix given a list of terms.
    The underlying datastructure is a numpy matrix of dimensions len(terms) x len(terms)
    
    Scoring options:
     - fuzz.ratio (default)
     - fuzz.partial_ratio
     - fuzz.token_sort_ratio
     - fuzz.token_set_ratio
    
    See this link for descriptions of the different distance metrics:
    https://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/
    """
    def __init__(self, terms, scorer=fuzz.ratio, matrix=None):
        """
        Create a FuzzyMatrix object from terms
        """
        self.terms = terms
        self.scorer = scorer
        self.matrix = None
        self.dict = {}
        for i in range(len(terms)):
            self.dict[terms[i]] = i
        
        if matrix == None:
            self.calc_matrix()
        else:
            self.matrix = matrix
    
    def get_term(self, index: int):
        if index >= len(self.terms):
            return None
        return self.terms[index]
    
    def score(self, term1, term2):
        if term1 not in self.dict or term2 not in self.dict:
            return None
        
        i = self.dict[term1]
        j = self.dict[term2]
        return self.matrix[i][j]
    
    def calc_matrix(self):
        """
        Calculate the similarity matrix using the similarity matrix defined in self.scorer
        """
        print("Building similarity matrix...", flush=True)
        size = len(self.terms)
        self.matrix = np.empty((size, size))
        for i in range(size):
            for j in range(i, size):
                similarity_score = self.scorer(self.terms[i], self.terms[j])
                self.matrix[i][j] = similarity_score
                self.matrix[j][i] = similarity_score

In [17]:
class Model:
    """
    Wrapper around k-means clustering and the similarity matrix FuzzMatrix
    """
    def __init__(self, terms, frequencies, scorer=fuzz.ratio):
        """
        terms         : 1-d array-like list of unique terms to cluster
        frequencies   : 1-d array-like list of frequencies/number of observations of each column name
                        Frequencies will be used as the weights in K-means clustering
        
        terms and frequencies must be the same length
        
        """
        
        assert len(terms) == len(frequencies), "terms and frequencies must be the same length"
        self.frequencies = frequencies
        self.fuzz_matrix = FuzzMatrix(terms, scorer)
        self.kmeans = None
        self.clusters = {}
        self.distances = []
        self.sorted_distances = []
    
    def k_means(self, num_clusters: int = 100):
        print("Running K-means clustering on {} terms with {} clusters...".format(len(self.fuzz_matrix.terms), num_clusters), flush=True)
        self.kmeans = KMeans(num_clusters).fit(self.fuzz_matrix.matrix, sample_weight=self.frequencies)
        
        # sort each terms into clusters -- key: label, value: [list of terms represented by index]
        self.clusters = {}
        for i in range(len(self.kmeans.labels_)):
            label = self.kmeans.labels_[i]
            if label not in self.clusters:
                self.clusters[label] = [i]
            else:
                self.clusters[label].append(i)
                
        # calculate mean distance for each centroid
        self.distances = self.calc_mean_distances(self.kmeans.labels_, self.kmeans.cluster_centers_, self.fuzz_matrix.matrix)
        self.sorted_distances = sorted([(i, self.distances[i]) for i in range(len(self.distances))], key=lambda x : x[1])
        
        print("Finished running k-means and computing mean cluster spread", flush=True)
            
    def calc_mean_distances(self, labels, centroids, data):
        """
        Calculate the average distance for each centroid and return a list of tuples (cluster_index, avg distance)
        
        Returns a list of length len(centroids)
        """
        distances = [0 for n in range(len(centroids))]
        counts = [0 for n in range(len(centroids))]
        
        
        for i in range(len(data)):
            row = data[i]
            label = labels[i]
            centroid = centroids[label]
            dist = np.linalg.norm(row - centroid)
            distances[label] += dist
            counts[label] += 1
        

        return np.array(distances) / np.array(counts)
        
    def view_clusters(self, top=10):
        """
        Print out the clusters with the lowest distance scores
        """
        for i in range(top):
            label = self.sorted_distances[i][0]
            distance = self.sorted_distances[i][1]
            cluster = self.clusters[label]
            
            print("Cluster", i + 1)
            print("  Mean distance:", distance)
            print("  Terms:", [self.fuzz_matrix.get_term(x) for x in cluster])
        

## Main code to run model

In [28]:
# Import csv file
csv = "columns_doe.csv"
df = pd.read_csv(csv)

# Count occurence of each column name
df_text = df.loc[df["columns_datatype"] == "Text"]
counts = df_text["column_name"].value_counts()

unique_column_names = counts.index.tolist()

### Run Levenshtein distance K-means model

In [29]:
# Create and run k-means model
model = Model(unique_column_names, counts.tolist())
model.k_means(100)

Building similarity matrix...
Running K-means clustering on 3499 terms with 100 clusters...
Finished running k-means and computing mean cluster spread


### View top X clusters with lowest mean distance to centroid

In [31]:
model.view_clusters(10)

Cluster 1
  Mean distance: 2.098996940364457e-14
  Terms: ['Year']
Cluster 2
  Mean distance: 78.33021134250498
  Terms: ['TOTAL STUDENTS With 2 or More Suspensions or Removals', 'BLACK STUDENTS With 2 or More Suspensions or Removals', 'MULTI-RACIAL STUDENTS With 2 or More Suspensions or Removals', '11 STUDENTS With 2 or More Suspensions or Removals', 'UNKNOWN STUDENTS With 2 or More Suspensions or Removals', 'ASIAN STUDENTS With 2 or More Suspensions or Removals', 'AMERICAN INDIAN/ALASKAN NATIVE STUDENTS With 2 or More Suspensions or Removals', 'HISPANIC STUDENTS With 2 or More Suspensions or Removals', '10 STUDENTS With 2 or More Suspensions or Removals', '12 STUDENTS With 2 or More Suspensions or Removals', 'WHITE STUDENTS With 2 or More Suspensions or Removals', '13 STUDENTS With 2 or More Suspensions or Removals', 'ELL STUDENTS With 2 or More Suspensions or Removals', 'GEN ED STUDENTS With 2 or More Suspensions or Removals', '08 STUDENTS With 2 or More Suspensions or Removals', '1

## Extract column data from json files and write to csv

This has already been done and saved to "columns.csv" and "columns_doe.csv"

In [ ]:
def get_department(dataset):
    """
    Given a dataset, return the department
    """
    domain_metadata = dataset["classification"]["domain_metadata"]
    if domain_metadata == None:
        return None
    
    department = "Dataset-Information_Agency"
    for d in domain_metadata:
        if d["key"] == department:
            return d["value"].strip()
        
    return None

In [ ]:
def get_columns(dataset):
    """
    Given a dictionary representing a dataset, where each of the json files
    in analysis/metadata is considered a list of datasets, retrieve a list of column details,
    where each column is represented as a dictionary with the following keys:
    {"column_name", "column_field_name", "column_type", "dataset", "department"}
    """
    department = get_department(dataset)
    dataset_name = dataset["resource"]["name"]

    columns_name = dataset["resource"]["columns_name"]
    columns_field_name = dataset["resource"]["columns_field_name"]
    columns_datatype = dataset["resource"]["columns_datatype"]
    
    columns = []
    for i in range(len(columns_name)):
        column = {}
        column["column_name" ] = columns_name[i]
        
        if i < len(columns_field_name):
            column["columns_field_name"] = columns_field_name[i]
        else:
            column["columns_field_name"] = None
        if i < len(columns_datatype):
            column["columns_datatype"] = columns_datatype[i]
        else:
            column["columns_datatype"] = None
        
        column["dataset"] = dataset_name
        column["department"] = department
    
        columns.append(column)
    
    return columns

In [26]:
## Find all datasets from DOE
def extract_column_data(json_filenames, save_as, department=None):
    columns = []
    for file in json_filenames:
        file_json = None
        with open("metadata\\" + file) as f:
            file_json = json.load(f)

        for dataset in file_json:
            dpt = get_department(dataset)
            if department == None:
                columns += get_columns(dataset)
            elif dpt == department:
                columns += get_columns(dataset)

    df = pd.DataFrame(columns)
    df.to_csv(save_as)
    return df

In [27]:
## Get list of json filenames in the metadata dir
cwd = os.getcwd()
metadata_dir = cwd + "\metadata"
json_filenames = [x for x in os.listdir(metadata_dir) if x[-4:] == "json"]

extract_column_data(json_filenames, "columns_test.csv", department="Department of Education (DOE)")

,column_name,columns_field_name,columns_datatype,dataset,department
0,SAT Writing Avg. Score,sat_writing_avg_score,Text,2012 SAT Results,Department of Education (DOE)
1,SCHOOL NAME,school_name,Text,2012 SAT Results,Department of Education (DOE)
2,SAT Math Avg. Score,sat_math_avg_score,Text,2012 SAT Results,Department of Education (DOE)
3,Num of SAT Test Takers,num_of_sat_test_takers,Text,2012 SAT Results,Department of Education (DOE)
4,DBN,dbn,Text,2012 SAT Results,Department of Education (DOE)
...,...,...,...,...,...
18148,Description of Code,description_of_code,Text,2014-15 Discharge Reporting By Code - HS,Department of Education (DOE)
18149,Total Enrolled Students,total_enrolled_students,Text,2014-15 Discharge Reporting By Code - HS,Department of Education (DOE)
18150,Count of Students,count_of_students,Text,2014-15 Discharge Reporting By Code - HS,Department of Education (DOE)
18151,Code Type,code_type,Text,2014-15 Discharge Reporting By Code - HS,Department of Education (DOE)
